In [1]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
import joblib

# Mesurer le temps de chargement des données
start_time = time.time()
df = pd.read_csv('Dataset (1).csv')
print(f"Temps de chargement des données: {time.time() - start_time:.2f} secondes")

# Utiliser un échantillon des données (20% des données)
df_sample = df.sample(frac=0.2, random_state=42)  # Utiliser 20% des données

# Prétraitement des données
start_time = time.time()
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X = encoder.fit_transform(df_sample[['game_slug', 'discipline_title', 'event_title', 'participant_type', 'game_season', 'game_year']])
y = df_sample['country_name']
print(f"Temps de prétraitement: {time.time() - start_time:.2f} secondes")

# Appliquer RandomOverSampler pour équilibrer les classes
start_time = time.time()
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)
print(f"Temps de rééchantillonnage: {time.time() - start_time:.2f} secondes")

# Diviser les données rééchantillonnées en ensembles d'entraînement et de test
start_time = time.time()
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(f"Temps de division des données: {time.time() - start_time:.2f} secondes")

# Entraîner le modèle sur les données rééchantillonnées
start_time = time.time()
new_model = RandomForestClassifier(n_estimators=50, random_state=42)  # Limiter le nombre d'arbres pour les tests
new_model.fit(X_train, y_train)
print(f"Temps d'entraînement du modèle: {time.time() - start_time:.2f} secondes")

# Prédire les valeurs de l'ensemble de test
start_time = time.time()
y_pred = new_model.predict(X_test)
print(f"Temps de prédiction: {time.time() - start_time:.2f} secondes")

# Évaluer les performances du modèle
print(f"Précision: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Sauvegarder le nouveau modèle et l'encodeur
joblib.dump(new_model, 'modele_prediction_pays_test.pkl')
joblib.dump(encoder, 'encodeur_test.pkl')


Temps de chargement des données: 0.19 secondes
Temps de prétraitement: 0.11 secondes
Temps de rééchantillonnage: 5.37 secondes
Temps de division des données: 6.11 secondes
Temps d'entraînement du modèle: 143.74 secondes
Temps de prédiction: 4.04 secondes
Précision: 0.5607033006033361
                                       precision    recall  f1-score   support

                          Afghanistan       0.78      0.73      0.75       321
                              Albania       0.58      0.78      0.67       327
                              Algeria       0.66      0.34      0.45       305
                       American Samoa       0.88      0.59      0.71       347
                              Andorra       0.46      0.48      0.47       300
                               Angola       0.72      0.60      0.65       332
                  Antigua and Barbuda       0.55      0.71      0.62       339
                            Argentina       0.38      0.25      0.30       330
   

['encodeur_test.pkl']

In [ ]:
import joblib
import pandas as pd
import time

# Charger le nouveau modèle sauvegardé
start_time = time.time()
new_model = joblib.load('modele_prediction_pays_test.pkl')
print(f"Temps de chargement du modèle: {time.time() - start_time:.2f} secondes")

# Charger l'encodeur sauvegardé
start_time = time.time()
encoder_pays = joblib.load('encodeur_test.pkl')
print(f"Temps de chargement de l'encodeur: {time.time() - start_time:.2f} secondes")

# Exemple de nouvelles données à prédire (doit être prétraité de la même manière que les données d'entraînement)
nouvelles_donnees_pays = pd.DataFrame([
    ['tokyo-2020', 'Diving', "Men's 10m Platform", 'Athlete', 'Summer', 2020],
    ['tokyo-2020', 'Athletics', "Men's 100m", 'Athlete', 'Summer', 2020],
    ['tokyo-2020', 'Swimming', "Women's 200m Freestyle", 'Athlete', 'Summer', 2020]
], columns=['game_slug', 'discipline_title', 'event_title', 'participant_type', 'game_season', 'game_year'])

# Encoder les nouvelles données
start_time = time.time()
nouvelles_donnees_pays_enc = encoder_pays.transform(nouvelles_donnees_pays)
print(f"Temps d'encodage des nouvelles données: {time.time() - start_time:.2f} secondes")

# Faire des prédictions
start_time = time.time()
predictions_pays = new_model.predict(nouvelles_donnees_pays_enc)
print(f"Temps de prédiction des nouvelles données: {time.time() - start_time:.2f} secondes")

nouvelles_donnees_pays['predicted_country'] = predictions_pays
print(nouvelles_donnees_pays)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Définir les hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Rechercher les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print(f"Meilleurs hyperparamètres : {grid_search.best_params_}")
best_model = grid_search.best_estimator_

# Évaluer le modèle optimisé
y_pred = best_model.predict(X_test)
print(f"Précision après optimisation : {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Sauvegarder le modèle optimisé
joblib.dump(best_model, 'modele_optimise_prediction_pays_test.pkl')


In [ ]:
from sklearn.model_selection import cross_val_score

# Validation croisée
scores = cross_val_score(RandomForestClassifier(n_estimators=50, random_state=42), X, y, cv=5)
print(f"Scores de validation croisée : {scores}")
print(f"Score moyen : {scores.mean()}")

